In [6]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
import os # getting filepath
import copy
import json
import matplotlib.pyplot as plt
import math
import sqlite3


In [7]:

# def main():
#     load_bool = input("Are you loading a behaviour csv? (y/n): ")
#     # NO, I AM NOT LOADING
#     if load_bool == 'n':
#         # get list of files
#         dir_name = input("Directory name: ")
#         contents = os.listdir(dir_name) # the filenames
#         # behaviour = int(input("Requested behaviour (number): "))
#         # behaviour_table = pd.DataFrame(data=None, columns=['behaviour', 'applicablefiles','indices'])
#         behaviour_table = pd.DataFrame({'behaviour': range(0,27)})
#         behaviour_table['applicablefiles'] = np.empty((len(behaviour_table), 0)).tolist()
#         behaviour_table['indices'] = np.empty((len(behaviour_table), 0)).tolist()
#             # behaviour = number, applicablefiles = array of filenames, 
#             # indices = array of tuples of tuples with order respect to filename order
#         # for each file, look into behavior column and gather information
        
#         # json table
#         table = dict()

#         for i in contents:
#             i = os.path.join(os.path.relpath(dir_name), i)
#             data = pd.read_csv(i, sep=",", 
#                             header=0, names=['depth', 'x', 'y', 'z', 'behaviour', 'type', 'place'],
#                             low_memory=False)
            
            
            
#             indices_arr = []
            # append filenames
#             file_behaviours = data.behaviour.unique() # NOTE: NOT SURE ABOUT BEHAVIOUR
#             for j in behaviour_table['behaviour']:
#                 if j in file_behaviours:
#                     behaviour_table.iloc[j+1][1] = behaviour_table.iloc[j+1][1].append(contents[i]) # NOTE: UNSURE ABOUT BEHAVIOUR
#             for ji, jr in behaviour_table:
#                 if jr['behaviour'] in file_behaviours:
#                     jr['behaviour'] = 
            # append indices for a single file
#                 for idx_row, row in data.iterrows(): 
#                     start_idx = -1
#                     end_idx = -2
#                     recording = False
#                     if recording == False and row['behaviour'] == j:
#                         start_idx = idx_row
#                         recording = True
#                     elif recording == True and row['behaviour'] != j:
#                         end_idx = idx_row-1
#                         recording = False
#                     elif start_idx == -1 and end_idx == -2:
#                         break
#                     indices_arr.append([start_idx, end_idx])
#                 print(indices_arr)
                # set 'cell' = indices_arr
                # behaviour_table[behaviour_table['behaviour' == j]]['indices'] = indices_arr
#                 behaviour_table.iloc[j][2] = indices_arr # have to use iloc
                # print(behaviour_table.head())
        
        # save behaviour table as csv
#         end_filename = "behaviourtable" + dir_name
#         behaviour_table.to_csv(end_filename, sep=",", header=['behaviour', 'applicablefiles', 'indices'])
        # print(behaviour_table.head())
    
    
    
#     # YES, I AM LOADING
#             for fi, fr in data.iterrows():
#                 table[i] = dict()
#         print(table)
#     elif load_bool == 'y':
#         print("uh")
#     # YOU SHMUCKED UP
#     else:
#         print("Something went wrong.")

# if __name__ == "__main__":
#     main()

In [8]:
def find_ranges(iterable):
    """Yield range of consecutive numbers."""
    for group in mit.consecutive_groups(iterable):
        group = list(group)
        if len(group) == 1:
            yield group[0]
        else:
            yield group[0], group[-1]

In [13]:
import pandas as pd
import numpy as np
import os
import more_itertools as mit
import sqlite3

conn = sqlite3.connect('fursealdata.db')
c = conn.cursor()
c.execute('''DROP TABLE fursealdata''')
c.execute('''CREATE TABLE fursealdata
             (behaviour integer, filename text, range text, depth blob, x blob, 
             y blob, z blob, type blob, place blob)''')
conn.commit()

# iterate through files and insert information into the db file
dir_name = input("Directory name: ")
contents = os.listdir(dir_name) # the filenames

for i in contents:
    i = os.path.join(os.path.relpath(dir_name), i)
    if i.lower().endswith(('.csv')) == False:
        print('hi')
        continue
    # now nothing but CSVs should be getting by
    data = pd.read_csv(i, sep=",", header=0, names=['depth', 'x', 'y', 'z', 'behaviour', 'type', 'place'], low_memory=False)
    # print(data)
    for behaviour in range(0, 27):
        temp = data.loc[data['behaviour'] == behaviour]
#         print(b, '=' * 20)
        if len(temp) > 0:
#             print(i)
            indexes = temp.index.tolist()
            groups = list(find_ranges(indexes))
            for r in groups:
#                 print(r)
#                 print(data['x'].iloc[groups[0]])
                x = data.iloc[r[0]:r[1]]['x']
                y = data.iloc[r[0]:r[1]]['y']
                z = data.iloc[r[0]:r[1]]['z']
                depth = data.iloc[r[0]:r[1]]['depth']
                t = data.iloc[r[0]:r[1]]['type']
                place = data.iloc[r[0]:r[1]]['place']
#                 print(type(list(place)))
                c.execute("INSERT INTO fursealdata VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", 
                          (behaviour, i, str(r), str(list(depth)), str(list(x)), 
                           str(list(y)), str(list(z)), str(list(t)), str(list(place))))
            conn.commit()
conn.close()

Directory name: cleandata_copy
hi


In [14]:
conn = sqlite3.connect('fursealdata.db')
c = conn.cursor()

files_and_ranges = []

for row in c.execute("SELECT * FROM fursealdata WHERE behaviour = 4"):
    files_and_ranges.append(row)
    
conn.close()
# print(files_and_ranges[0], files_and_ranges[1])

In [33]:
conn = sqlite3.connect('fursealdata.db')
c = conn.cursor()

num_ranges = len(files_and_ranges) # number of ranges in the file
temp = math.ceil(math.sqrt(num_ranges))
k = 0
for file in files_and_ranges: # 'file' is a misnomer; it's a single range out of all of them
    filename = file[1] # one filename per index (can be repeated amongst files_and_ranges)
    filename_for_fig = filename.replace(".csv", "")
    filename_for_fig = filename_for_fig.replace("cleandata_copy/", "")
    b = file[0] # one behavior per index (will be repeated amongst files_and_ranges)
#     plt.figure(dpi=300) # one figure per range
    range_for_axis = file[2] # 
    depth_range = file[3]
    x_range = file[4]
    y_range = file[5]
    z_range = file[6]
#         t_range = file[7]
#         place_range = file[8]
    # grab data from database
    for char in '[,]()':
        range_for_axis = range_for_axis.replace(char,"")
        x_range = x_range.replace(char,"")
        y_range = y_range.replace(char, "")
        z_range = z_range.replace(char, "")
        depth_range = depth_range.replace(char, "")

    range_for_axis = list(map(int, range_for_axis.split()))
    x_range = list(map(float, x_range.split()))
    y_range = list(map(float, y_range.split()))
    z_range = list(map(float, z_range.split()))
    depth_range = list(map(float, depth_range.split()))
    k += 1   # increment index for subplot   
#     plt.subplot(2, temp, k)  # one subplot per range in file
    plt.figure(dpi=300, figsize=[8.5, 11])
    fig_name = str(b) + " " + filename_for_fig + str(range_for_axis)
    
    plt.plot(range(range_for_axis[0], range_for_axis[1]), x_range, label='x', color='red')
    plt.plot(range(range_for_axis[0], range_for_axis[1]), y_range, label='y', color='blue')
    plt.plot(range(range_for_axis[0], range_for_axis[1]), z_range, label='z', color='green')
    plt.plot(range(range_for_axis[0], range_for_axis[1]), depth_range, label='depth', color='magenta')
    plt.title(fig_name)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    
#     plt.show()
    fig_name += '.png'
    plt.savefig(fname=fig_name, transparent=True, format='png')
#     print(k)
    plt.close()
            
        
conn.close()